In [1]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
import numpy as np 
import scipy.stats as stats
from collections import Counter
import matplotlib.pyplot as plt
import umap
import matplotlib
import mygene
%matplotlib inline
import pickle
import sklearn
import random
import os
import time
seed = 42
np.random.seed(seed)
random.seed(seed)
import multiprocessing as mp
from tqdm import tqdm

import scanpy as sc
import torch

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans
from xgboost import XGBClassifier

plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-dark-palette')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

import openai
from dotenv import load_dotenv
load_dotenv('/mnt/home/pandavis/.env')
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
pwd

'/mnt/ufs18/home-230/pandavis/cellular_resilience/CellResilienceModel/codebase/notebooks/genePT'

In [3]:
adata = sc.read_10x_h5('../../../data/BreastCancer10xGenomics_Rep1/Xenium_FFPE_Human_Breast_Cancer_Rep1_cell_feature_matrix.h5')
adata

AnnData object with n_obs × n_vars = 167782 × 313
    var: 'gene_ids', 'feature_types', 'genome'

In [5]:
sampled_adata = sc.read_h5ad("sample_aorta_data_updated.h5ad")
sampled_adata

/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 9625 × 23331
    obs: 'patient', 'status', 'n_counts', 'n_counts_normalized', 'n_counts_normalized_log', 'cell_id', 'patient_rename', 'Unnamed: 0', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'stim', 'integrated_snn_res.0.6', 'seurat_clusters', 'celltype', 'celltype2', 'cellcycle', 'split_value', 'phenotype'
    var: 'ensembl_id'
    uns: 'log1p', 'neighbors', 'patient_colors', 'pca', 'status_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [6]:
sampled_adata.obs

,patient,status,n_counts,n_counts_normalized,n_counts_normalized_log,cell_id,patient_rename,Unnamed: 0,orig.ident,nCount_RNA,nFeature_RNA,stim,integrated_snn_res.0.6,seurat_clusters,celltype,celltype2,cellcycle,split_value,phenotype
index,,,,,,,,,,,,,,,,,,,
CCCAACTAGTCACTAC.1ATAA5,affected_5,affected,8016.0,10000.0,2925.760292,CCCAACTAGTCACTAC.1,ATAA5,CCCAACTAGTCACTAC.1_8,ATAA5,8016.0,2583.0,ATAA,7.0,7.0,MonoMaphDC,M2like1,G1,CCCAACTAGTCACTAC.1,Ascending w/ root
TCGGGCACACGCAAAG.1ATAA6,affected_6,affected,4563.0,10000.0,2144.931018,TCGGGCACACGCAAAG.1,ATAA6,TCGGGCACACGCAAAG.1_9,ATAA6,4563.0,1397.0,ATAA,1.0,1.0,Tcell,CD4_EM_rest,G1,TCGGGCACACGCAAAG.1,Ascending to descending
TCGAGGCGTCATACTG.1ATAA1,affected_1,affected,1654.0,10000.0,1768.070160,TCGAGGCGTCATACTG.1,ATAA1,TCGAGGCGTCATACTG.1_4,ATAA1,1653.0,770.0,ATAA,3.0,3.0,Tcell,CD8_active,G1,TCGAGGCGTCATACTG.1,Ascending only
TTGACTTGTGCGCTTG.1ATAA2,affected_2,affected,3067.0,10000.0,2514.338987,TTGACTTGTGCGCTTG.1,ATAA2,TTGACTTGTGCGCTTG.1_5,ATAA2,3062.0,1430.0,ATAA,6.0,6.0,Fibroblast,Fibroblast1,G1,TTGACTTGTGCGCTTG.1,Ascending only
CGAGTGCAGTAAGACT.1ATAA7,affected_7,affected,6630.0,10000.0,2920.036199,CGAGTGCAGTAAGACT.1,ATAA7,CGAGTGCAGTAAGACT.1_10,ATAA7,6630.0,2294.0,ATAA,2.0,2.0,SMC1,Contractile SMC,S,CGAGTGCAGTAAGACT.1,Ascending w/ root
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTAGACCACCAAATC.1ATAA5,affected_5,affected,622.0,10000.0,1095.416653,GTTAGACCACCAAATC.1,ATAA5,GTTAGACCACCAAATC.1_8,ATAA5,621.0,353.0,ATAA,20.0,20.0,MonoMaphDC,Monocyte,G1,GTTAGACCACCAAATC.1,Ascending w/ root
CGAGGAAAGATAGCTA.1ATAA8,affected_8,affected,7387.0,10000.0,2313.681128,CGAGGAAAGATAGCTA.1,ATAA8,CGAGGAAAGATAGCTA.1_11,ATAA8,7386.0,1788.0,ATAA,8.0,8.0,NK,NK,G1,CGAGGAAAGATAGCTA.1,Ascending w/ root
TAAGTCGAGTCACACT.1ATAA6,affected_6,affected,3605.0,10000.0,2152.200651,TAAGTCGAGTCACACT.1,ATAA6,TAAGTCGAGTCACACT.1_9,ATAA6,3605.0,1284.0,ATAA,3.0,3.0,Tcell,T_HSP,S,TAAGTCGAGTCACACT.1,Ascending to descending


In [ ]:
def gene_seq_embedding(X, gene_names, prompt_prefix=""):
    n_genes = X.shape[1]
    get_test_array = []
    for cell in (X):
        zero_indices = (np.where(cell==0)[0])
        gene_indices = np.argsort(cell)[::-1]
        filtered_genes = gene_indices[~np.isin(gene_indices, list(zero_indices))]
        get_test_array.append(np.array(gene_names[filtered_genes])) 
    get_test_array_seq = [prompt_prefix+' '.join(x) for x in get_test_array]
    return(get_test_array_seq)

def get_gpt_embedding(text, model="text-embedding-ada-002"):
    time.sleep(1 / 40)
    text = text.replace("\n", " ")
    return np.array(openai.Embedding.create(input = [text], model=model)['data'][0]['embedding'])

sample_cells_data = get_seq_embed_gpt(adata.X.toarray(),
                                    np.array(adata.var.index), 
        prompt_prefix = 'A cell with genes ranked by expression: ')

sampled_cell_gpt = []
for x in sample_cells_data:
    sampled_cell_gpt.append(get_gpt_embedding(x))
sampled_cell_gpt = np.array(sampled_cell_gpt)

In [4]:
with open("../../../data/BreastCancer10xGenomics_Rep1/exported_data/gene_embeddings.pickle", "rb") as fp:
    gene_embeddings = pickle.load(fp) 

In [5]:
gene_names = list(adata.var_names)  
EMBED_DIM = 1536                   
lookup_embed = np.zeros((len(gene_names), EMBED_DIM))

In [6]:
missing_genes = 0
for i, gene in enumerate(gene_names):
    if gene in gene_embeddings:
        lookup_embed[i, :] = gene_embeddings[gene].flatten()  # Add embedding if available
    else:
        missing_genes += 1  # Count missing genes

print(f"Missing {missing_genes} out of {len(gene_names)} genes.")

Missing 7 out of 313 genes.


In [7]:
#cell_embeddings = np.dot(adata.X, lookup_embed) / len(gene_names)
adata_X_gpu = torch.tensor(adata.X.toarray(), device='cuda', dtype=torch.float32)
lookup_embed_gpu = torch.tensor(lookup_embed, device='cuda', dtype=torch.float32)
cell_embeddings = torch.matmul(adata_X_gpu, lookup_embed_gpu) / len(gene_names)
cell_embeddings = cell_embeddings.cpu().numpy()

In [8]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

In [ ]:
import faiss
import torch
import numpy as np
from sklearn.metrics import silhouette_score

# Move data to GPU for faster computation
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Example: Assuming 'cell_embeddings' is available
# Converting embeddings to float32 required by Faiss
cell_embeddings = torch.tensor(cell_embeddings, dtype=torch.float32).to(device)
cell_embeddings_np = cell_embeddings.cpu().numpy()  # Faiss requires NumPy arrays

# Number of clusters to test
cluster_range = range(2, 21)  # Testing 2 to 20 clusters
silhouette_scores = {}

# Loop through each cluster size
for n_clusters in tqdm(cluster_range):
    d = cell_embeddings.shape[1]  # Number of features (300)

    # Initialize Faiss KMeans
    kmeans = faiss.Kmeans(d, n_clusters, niter=300, verbose=False, gpu=torch.cuda.is_available())

    # Train the KMeans model
    kmeans.train(cell_embeddings_np)

    # Predict cluster labels
    D, predicted_labels = kmeans.index.search(cell_embeddings_np, 1)  # Search for nearest cluster
    predicted_labels = predicted_labels.flatten()

    # Compute silhouette score
    subset_size = 10000
    indices = np.random.choice(cell_embeddings_cpu.shape[0], subset_size, replace=False)

    score = silhouette_score(cell_embeddings_cpu[indices], predicted_labels_cpu[indices])
    silhouette_scores[n_clusters] = score
    print(f"Clusters: {n_clusters}, Silhouette Score: {score}")

# Find the number of clusters with the highest silhouette score
best_n_clusters = max(silhouette_scores, key=silhouette_scores.get)
best_score = silhouette_scores[best_n_clusters]

# Output the results
best_n_clusters, best_score, silhouette_scores

/tmp/ipykernel_124402/3012585895.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cell_embeddings = torch.tensor(cell_embeddings, dtype=torch.float32).to(device)
  0%|                                                                                                                        | 0/19 [00:00<?, ?it/s]/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  5%|█████▉                                                                                                          | 1/19 [00:03<01:08,  3.78s/it]

Clusters: 2, Silhouette Score: 0.38818538188934326


/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 11%|███████████▊                                                                                                    | 2/19 [00:07<01:06,  3.93s/it]

Clusters: 3, Silhouette Score: 0.3901044428348541


/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 16%|█████████████████▋                                                                                              | 3/19 [00:11<01:03,  3.95s/it]

...

Clusters: 14, Silhouette Score: 0.3886895477771759


/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 74%|█████████████████████████████████████████████████████████████████████████████████▊                             | 14/19 [01:16<00:33,  6.73s/it]

Clusters: 15, Silhouette Score: 0.3870384395122528


/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 79%|███████████████████████████████████████████████████████████████████████████████████████▋                       | 15/19 [01:22<00:26,  6.53s/it]

Clusters: 16, Silhouette Score: 0.3876637816429138


In [14]:
cell_embeddings_cpu = cell_embeddings.cpu().numpy()
predicted_labels_cpu = predicted_labels.cpu().numpy()

indices = np.random.choice(cell_embeddings_cpu.shape[0], subset_size, replace=False)
silhouette = silhouette_score(cell_embeddings_cpu[indices], predicted_labels_cpu[indices])
print("Silhouette Score:", silhouette)

/mnt/home/pandavis/anaconda3/envs/pytorch/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Silhouette Score: 0.39223725


In [13]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score

# Evaluate clustering quality
ari = adjusted_rand_score(adata.obs['cell_type'], predicted_labels)  # ARI score
ami = adjusted_mutual_info_score(adata.obs['cell_type'], predicted_labels)  # AMI score

print(f"ARI: {ari:.2f}, AMI: {ami:.2f}, Silhouette: {silhouette:.2f}")


KeyError: 'cell_type'

{'ABCC11': array([-0.01112988, -0.01357845, -0.00782464, ..., -0.03394276,
        -0.03054308, -0.02513329]),
 'ACKR1': array([-0.01863805,  0.00190632, -0.02583479, ..., -0.03771145,
        -0.03153504, -0.00384325]),
 'ACTA2': array([-0.03217858, -0.01856354, -0.02966421, ..., -0.0100441 ,
         0.00470107, -0.02268282]),
 'ACTG2': array([-0.03632329, -0.02169894, -0.02253047, ..., -0.02819279,
         0.01742251, -0.01742251]),
 'ADAM9': array([-0.03382774, -0.00380695, -0.018122  , ..., -0.02267574,
        -0.00824452, -0.00828434]),
 'ADGRE5': array([-0.0259791 , -0.0018996 , -0.03338868, ..., -0.0416447 ,
        -0.01097112,  0.00294462]),
 'ADGRL4': array([-0.02583771, -0.0027584 , -0.02304623, ..., -0.03934529,
        -0.01297839, -0.00303623]),
 'ADH1B': array([-0.02072373, -0.00851339, -0.01953915, ..., -0.04772187,
        -0.0073223 ,  0.01629781]),
 'ADIPOQ': array([-0.02689763, -0.00444722, -0.00431661, ..., -0.01875333,
        -0.01839166, -0.00717315]),
 'AEBP